# Supplemental Code for "Powerful batch conformal prediction for classification" by U. Gazin, R. Heller, E. Roquain and A. Solari.

In [1]:
import random
import numpy as np
import statsmodels as sm
import matplotlib.pyplot as plt
from math import factorial
import scipy.stats as stat
import pandas as pd


import sklearn
from sklearn.datasets import fetch_openml 
from sklearn.ensemble import RandomForestClassifier,IsolationForest
from sklearn.svm import OneClassSVM

from time import perf_counter

import seaborn as sns

## Disclaimer on the packages

The packages blackbox.py and nn.py in folder source are  based on the codes by Ariane Marandon from the paper "Selecting informative conformal prediction sets with false coverage rate control" by Gazin, Heller, Marandon, Roquain (see https://github.com/arianemarandon/infoconf/blob/main/source/nn.py) itself inspired by the code from the paper "Classification with Valid and Adaptive Coverage" by Romano, Sesia and Candès (Neurips 2020) (See https://github.com/msesia/arc).

In [2]:
import source.black_boxes as bb

2025-03-09 19:08:46.714752: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-09 19:08:46.724323: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-09 19:08:46.735533: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-09 19:08:46.738880: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-09 19:08:46.747352: I tensorflow/core/platform/cpu_feature_guar

In [3]:
from GeneralBatchCode import *

In [4]:
#Setting colors for Graphics

colors_blindness = sns.color_palette("colorblind")

colorA=colors_blindness[1]
colorB=colors_blindness[2]
color_Rose=colors_blindness[4]
plt.rcParams['text.usetex'] = True
plt.rc('text.latex', preamble=r'\usepackage{bm} \usepackage{amsfonts}')

In [5]:
SimesStorey1_2=AdaptiveSimesFunctionGenerator(1/2)
SimesStorey5_100=AdaptiveSimesFunctionGenerator(0.05) 
Quantile1_2=QuantileSimesFunctionGenerator(1/2)
Procedure=['Simes', 'Bonf.',  r"Storey $\lambda=1/2$",r"Storey $\lambda=5\%$",'Median','Fisher']
TabFunction=[SimesFunction,BonfFunction,SimesStorey1_2,SimesStorey5_100,Quantile1_2,FisherFunction]

In [6]:
Storey1_2=Storeym0FunctionGenerator(1/2)
Storey2_3=Storeym0FunctionGenerator(2/3)

# Code for nice illustration 
You can use these codes only when correct labels are known.

In [7]:
#To Obtain Nices Pictures

def VizualisationPredictionSet(alpha,pVal,I,Xtest,ytest,Ncal,TabFunction,NameFunction,GeneralQuantileFunction,nClass,Label,See,Compt='0',SaveFigs=False):
    nI=len(I)
    nProc=len(TabFunction)
    
    YPredict=np.append(np.zeros(nI),NameFunction)
    for k in range(nI):
        YPredict[k]=Label[int(ytest[I[k]])]
    
    y=np.zeros(nI)
    NbCompt=0
    nBoucle=nClass**nI
    QuantileFunction=GeneralQuantileFunction(alpha,nClass,Ncal,nI,N=10**3)
    nPredict=0
    pValI=pVal[I]
    while NbCompt<nBoucle:
        SeeCurrenty=False
        
        for i in range(nProc):
            Function=TabFunction[i]
            
            if not SeeCurrenty and IndicatorPredictionSet(y, pValI,Function,QuantileFunction,nClass):
                SeeCurrenty=True
                
                ToAdd=[]
                for k in range(nI):
                    
                    ToAdd.append(Label[int(y[k])])
                for k in range(nProc):
                    ToAdd.append("{:.3f}".format(min(TabFunction[k]( pValI[np.arange(nI),y.astype(int)]),1)))
                
                YPredict=np.vstack([YPredict,ToAdd])
                nPredict+=1



        y=CompteurYclass(y,nClass)

        NbCompt+=1
    
    
    
    
    
    plt.figure(figsize=(11,3))

    
    for i in range(nI):
        plt.subplot(1,nI,i+1)

        See(Xtest[I[i]])
        plt.gca().set_title(Label[int(ytest[I[i]])],fontsize=15)

    if SaveFigs:
        plt.savefig('Images'+Compt+'.pdf',format='pdf')

    plt.show()


    
    
    
    
    
    if nPredict>0:
        DataFramePval=pd.DataFrame(YPredict[1:],columns=YPredict[0])
        DataFramePval.sort_values(by=['Bonf.','Simes'], inplace=True,ascending=[True,True])
        if SaveFigs:
            
            DataFramePval.to_latex(buf='pVal'+Compt+'.txt',index=False,formatters={"name": str.upper})


        return DataFramePval
    else:
        return  pd.DataFrame([],columns=YPredict)
    
    
    
def VizualisationLowerBoundSet(alpha,DataFramePval,LenI,NameFunction,Label,Compt='0',SaveFigs=False):
    
    nProc=len(NameFunction)
    nLabel=len(Label)
    
    TabBounds=np.ones((nLabel+1,nProc)).astype(str)
    for k in range(nProc):
        DFProc=DataFramePval[DataFramePval[NameFunction[k]]>str(alpha)]
        YPredictProc=DFProc.iloc[:,:LenI].to_numpy()
        nPredict=len(YPredictProc)
        Tot=PostHocBoundsLabels(YPredictProc,Label,np.arange(LenI).astype(int))
        Low=Tot[0]
        Up=Tot[2]
    
        TabBounds[nLabel][k]=nPredict
        for i in range(nLabel):
            TabBounds[i][k]=("{:.0f}".format((Low[i]))+' ; '+"{:.0f}".format((Up[i])))
            
    LabelAdd=np.append(Label,'Size')     
    DataFrameBounds=pd.DataFrame(TabBounds, index = LabelAdd, columns =NameFunction)
        
        
        
    if SaveFigs:
            
        DataFrameBounds.to_latex(buf='Bounds'+Compt+'.txt',index=True,formatters={"name": str.upper})


    return DataFrameBounds

# Experiments

We work with only one batch and provide a batch prediction region and bounds.

## USPS dataset

In [8]:
# load dataset

datasetUSPS = fetch_openml(name='USPS',  as_frame=False)
XUSPS = datasetUSPS.data
yUSPS = datasetUSPS.target.astype(float)
nClassUSPS=10
LabelUSPS=[str(i) for i in range(10)]

/home/ugazin/.local/lib/python3.10/site-packages/sklearn/datasets/_openml.py:303: UserWarning: Multiple active versions of the dataset matching the name usps exist. Versions may be fundamentally different, returning version 2.
  warn(
/home/ugazin/.local/lib/python3.10/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [9]:
#### To see images

def SeeUSPS(X):
    img=X.reshape(16,16)
    plt.imshow(img,cmap='gray')

In [ ]:
SeeUSPS(XUSPS[0])

In [ ]:
# Split the dataset between training set, calibration set and test set.

ncalUSPS=int(np.ceil(7290*2/3))
ntrainUSPS=7291-ncalUSPS

XUSPStrain=XUSPS[:ntrainUSPS]
yUSPStrain=(yUSPS[:ntrainUSPS]-1).astype(int)
XUSPSTestCal=XUSPS[ntrainUSPS:]
yUSPSTestCal=(yUSPS[ntrainUSPS:]-1).astype(int)
XUSPScal=XUSPS[ntrainUSPS:ntrainUSPS+ncalUSPS]
yUSPScal=(yUSPS[ntrainUSPS:ncalUSPS+ntrainUSPS]-1).astype(int)


XUSPStest=XUSPS[7291:]
yUSPStest=(yUSPS[7291:]-1).astype(int)

In [ ]:
# train a Support-Vector Classifier

ClassifieurUSPS_SVC=bb.SVC(random_state=5)
ClassifieurUSPS_SVC.fit(XUSPStrain,yUSPStrain)

In [ ]:
(pValUSPSFull,ncalUSPS)=FullCalibratedpvalues(ClassifieurUSPS_SVC,10,XUSPScal,yUSPScal,ncalUSPS,XUSPStest,2007)

In [ ]:
PredictionBonf=PredictionSet(0.01,pValUSPSFull,np.arange(5).astype(int),ncalUSPS,BonfFunction,AlphaLevel,10)
PredictionSimes=PredictionSet(0.01,pValUSPSFull,np.arange(5).astype(int),ncalUSPS,SimesFunction,AlphaLevel,10)
PredictionSimesStorey=PredictionSet(0.01,pValUSPSFull,np.arange(5).astype(int),ncalUSPS,SimesStorey1_2,AlphaLevel,10)
PredictionMedian=PredictionSet(0.01,pValUSPSFull,np.arange(5).astype(int),ncalUSPS,Quantile1_2,AlphaLevel,10)
PredictionFisher=PredictionSet(0.01,pValUSPSFull,np.arange(5).astype(int),ncalUSPS,FisherFunction,AlphaLevel,10)

In [ ]:
# To see the test dataset:
plt.figure(figsize=(11,3))
for i in np.arange(5).astype(int):
    plt.subplot(1,5,i+1)

    SeeUSPS(XUSPStest[i])

plt.show()

In [ ]:
print('Size of the batch prediction set with Bonf. procedure is: '+str(len(PredictionBonf)))
print('Prediction set with the Bonf. procedure:')
print(PredictionBonf)

BoundsBonf=PostHocBounds(PredictionBonf,10,range(5))
print('Lower bounds with Bonf. procedure :' +str(BoundsBonf[0]))
print('Upper bounds with Bonf. procedure :' +str(BoundsBonf[2]))

In [ ]:
print('Size of the batch prediction set with Simes procedure is: '+str(len(PredictionSimes)))
print('Prediction set with the Simes procedure:')
print(PredictionSimes)

BoundsSimes=PostHocBounds(PredictionSimes,10,range(5))
print('Lower bounds with Simes procedure :' +str(BoundsSimes[0]))
print('Upper bounds with Simes procedure :' +str(BoundsSimes[2]))

In [ ]:
print('Size of the batch prediction set with Storey procedure is: '+str(len(PredictionSimesStorey)))
print('Prediction set with the Storey procedure:')
print(PredictionSimesStorey)

BoundsSimesStorey=PostHocBounds(PredictionBonf,10,range(5))
print('Lower bounds with Storey procedure :' +str(BoundsSimesStorey[0]))
print('Upper bounds with Storey procedure :' +str(BoundsSimesStorey[2]))

In [ ]:
print('Size of the batch prediction set with Median procedure is: '+str(len(PredictionMedian)))
print('Prediction set with the Median procedure:')
print(PredictionMedian)

BoundsMedian=PostHocBounds(PredictionBonf,10,range(5))
print('Lower bounds with Median procedure :' +str(BoundsMedian[0]))
print('Upper bounds with Median procedure :' +str(BoundsMedian[2]))

In [ ]:
print('Size of the batch prediction set with Fisher procedure is: '+str(len(PredictionFisher)))
print('Prediction set with the Fisher procedure:')
print(PredictionFisher)

BoundsFisher=PostHocBounds(PredictionBonf,10,range(5))
print('Lower bounds with Fisher procedure :' +str(BoundsFisher[0]))
print('Upper bounds with Fisher procedure :' +str(BoundsFisher[2]))

In [ ]:
(pValClassUSPS,ncalClassUSPS)=ClassCalibratedpvalues(ClassifieurUSPS_SVC,nClassUSPS,XUSPScal,yUSPScal,ncalUSPS,XUSPStest,2007)

In [ ]:
pVal175=(VizualisationPredictionSet(0.05,pValClassUSPS,(np.arange(5)+175).astype(int),XUSPStest,yUSPStest,ncalClassUSPS,TabFunction,Procedure,AlphaLevel,nClassUSPS,LabelUSPS,SeeUSPS,'USPS_175'))

In [ ]:
pVal175

In [ ]:
VizualisationLowerBoundSet(0.05,pVal175,10,Procedure,LabelUSPS)

## CIFAR Dataset

In [ ]:
datasetCIFAR = fetch_openml(name='CIFAR_10', as_frame=False)
XALLCIFAR = datasetCIFAR.data
yALLCIFAR = datasetCIFAR.target.astype(float)

UsefulData=(yALLCIFAR==2)+(yALLCIFAR==3)+(yALLCIFAR==5)
UsefulIndices=np.arange(len(yALLCIFAR))[UsefulData]
yCIFAR=np.array(yALLCIFAR)[UsefulIndices]
XCIFAR=XALLCIFAR[UsefulData]
yCIFAR=yALLCIFAR[UsefulData]
yCIFAR[yCIFAR==2]=0
yCIFAR[yCIFAR==3]=1
yCIFAR[yCIFAR==5]=2

nClassCIFAR=3
LabelCIFAR=['Bird','Cat','Dog']

In [ ]:
len(yALLCIFAR)

In [ ]:
#### To see images

def SeeCIFAR(X):
    img = X.reshape(3,32,32).astype(int)
    plt.imshow(img.transpose(1,2,0))

In [ ]:
SeeCIFAR(XCIFAR[0])

In [ ]:
ntotCIFAR=len(yCIFAR)
nClassCIFAR=3
nTestCIFAR=3000
prop=2/3
ncalCIFAR=int(np.ceil((ntotCIFAR-nTestCIFAR)*prop))
ntrainCIFAR=int(ntotCIFAR-nTestCIFAR-ncalCIFAR)

print("ntrain="+str(ntrainCIFAR)+" ncal="+str(ncalCIFAR) )


XCIFARtrain=XCIFAR[:ntrainCIFAR]
yCIFARtrain=(yCIFAR[:ntrainCIFAR]).astype(int)


XCIFARcal=XCIFAR[ntrainCIFAR:ntrainCIFAR+ncalCIFAR]
yCIFARcal=(yCIFAR[ntrainCIFAR:ncalCIFAR+ntrainCIFAR]).astype(int)


XCIFARtest=XCIFAR[ncalCIFAR+ntrainCIFAR:]
yCIFARtest=(yCIFAR[ncalCIFAR+ntrainCIFAR:]).astype(int)

XCIFARtestcal=XCIFAR[ntrainCIFAR:]
yCIFARtestcal=yCIFAR[ntrainCIFAR:].astype(int)

In [ ]:
Classifieur_CIFAR=bb.CNNkeras(3,32)
Classifieur_CIFAR.fit(XCIFARtrain,yCIFARtrain)

In [ ]:
LenI=8
Cal=10000
UniI=np.arange(LenI).astype(int)

TotChoice=np.random.choice(len(yCIFARtestcal),size=LenI+Cal,replace=False)
I=TotChoice[:LenI]
J=TotChoice[LenI:]
XtestCurr=XCIFARtestcal[I]
ytestCurr=yCIFARtestcal[I]
XcalCurr=XCIFARtestcal[J]
ycalCurr=yCIFARtestcal[J]
    
(pValClassCIFAR,ncalClassCIFAR)=ClassCalibratedpvalues(Classifieur_CIFAR,nClassCIFAR,XcalCurr,ycalCurr,Cal,XtestCurr,LenI)
pvalPredictionSet=(VizualisationPredictionSet(0.1,pValClassCIFAR,UniI,XtestCurr,ytestCurr,ncalClassCIFAR,TabFunction,Procedure,AlphaLevel,nClassCIFAR,LabelCIFAR,SeeCIFAR,str(I)))

Bounds=(VizualisationLowerBoundSet(0.1,pvalPredictionSet,LenI,Procedure,LabelCIFAR,str(I)))

In [ ]:
pvalPredictionSet

In [ ]:
Bounds

# Large sample study of the procedure

We provide functions to obtain the coverage, a 'power' criteria, and the size of the prediction set when doing repeated experiments.
In the paper we display graphics of the size, which can be obtained on the file 'Illustration_AISTATS25.ipynb' in the folder 'Illustration'.


Running these codes on a standard laptop can take some hours or days if the number of $N$ of replication, the size of the array Alpha which contained all the level $\alpha$ we want to test, the number of class or the size of the batch are too big

In [ ]:
from LargeSampleCode import *

In [ ]:
AlphaViolin=np.array([0.01,0.05,0.1])
AlphaBis=np.linspace(0.01,0.1,50)
#np.save('AlphaBis.npy',AlphaBis)

In [ ]:
(TotCovCIFAR_2000_5,TotPowerCIFAR_2000_5,TotnPredictCIFAR_2000_5)=PowerAndCoverage3MethodsRandomized(AlphaBis,5,2000,XCIFARtestcal,yCIFARtestcal,ClassCalibratedpvalues,Classifieur_CIFAR,TabFunction,AlphaLevel,3,N=500)
#np.save('TotCovCIFAR_2000_5.npy',TotCovCIFAR_2000_5)
#np.save('TotPowerCIFAR_2000_5.npy',TotPowerCIFAR_2000_5)
#np.save('TotnPredictCIFAR_2000_5.npy',TotnPredictCIFAR_2000_5)

In [ ]:
(TotCovUSPS_700_3,TotPowerUSPS_700_3,TotnPredictUSPS_700_3)=PowerAndCoverage3MethodsRandomized(AlphaBis,3,700,XUSPSTestCal,yUSPSTestCal,ClassCalibratedpvalues,ClassifieurUSPS_SVC,TabFunction,AlphaLevel,10,N=500)
#np.save('TotCovUSPS_700_3.npy',TotCovUSPS_700_3)
#np.save('TotPowerUSPS_700_3.npy',TotPowerUSPS_700_3)
#np.save('TotnPredictUSPS_700_3.npy',TotnPredictUSPS_700_3)

# Unbalanced class :  Large sample study of the procedure

## Cifar: Calibration with everyone, test only with cat and dog

In [ ]:
ClassifiedCIFAR=(np.where(yCIFARtestcal==0)[0],(np.where(yCIFARtestcal==1)[0]),np.where(yCIFARtestcal==2)[0])


In [ ]:
AlphaTest=[0.1]
TabFunctionTest=[SimesFunction,BonfFunction]

In [ ]:
PowerAndCoverage_Full_v_Class(AlphaTest,3,100,np.array([0.3,0.3,0.3]),np.array([0.5,0.5,0]),XCIFARtestcal,yCIFARtestcal,ClassifiedCIFAR,Classifieur_CIFAR,TabFunctionTest,AlphaLevel,3,N=10**1)

In [ ]:
(TotCoverage3,TotPower3,TotnPredict3)=PowerAndCoverage_Full_v_Class(AlphaViolin,5,2000,np.array([0.3,0.3,0.3]),np.array([0.5,0.6,0]),XCIFARtestcal,yCIFARtestcal,ClassifiedCIFAR,Classifieur_CIFAR,TabFunction,AlphaLevel,3,N=10**3)

In [ ]:
#np.save('TotCovCIFAR_2000_03_03_03_5_05_06_00_N1000.npy',TotCoverage3)
#np.save('TotPowerCIFAR_2000_03_03_03_5_05_06_00_N1000.npy',TotPower3)
#np.save('TotnPredictCIFAR_2000_03_03_03_5_05_06_00_N1000.npy',TotnPredict3)

In [ ]:
(TotCoverage_2,TotPower_2,TotnPredict_2)=PowerAndCoverage_Full_v_Class(AlphaViolin,5,2000,np.array([0.1,0.3,0.6]),np.array([0.9,0.2,0.2]),XCIFARtestcal,yCIFARtestcal,ClassifiedCIFAR,Classifieur_CIFAR,TabFunction,AlphaLevel,3,N=10**3)

In [ ]:
#np.save('TotCovCIFAR_2000_01_03_06_5_09_02_02_N1000.npy',TotCoverage_2)
#np.save('TotPowerCIFAR_2000_01_03_06_5_09_02_02_N1000.npy',TotPower_2)
#np.save('TotnPredictCIFAR_2000_01_03_06_5_09_02_02_N1000.npy',TotnPredict_2)

In [ ]:
AlphaViolin=[0.01,0.05,0.1]


In [ ]:
(TotCoverage_AlphaBis,TotPower_AlphaBis,TotnPredict_AlphaBis)=PowerAndCoverage_Full_v_Class(AlphaBis,5,2000,np.array([0.1,0.3,0.6]),np.array([0.9,0.2,0.2]),XCIFARtestcal,yCIFARtestcal,ClassifiedCIFAR,Classifieur_CIFAR,TabFunction,AlphaLevel,3,N=10**3)

In [ ]:
#np.save('TotCovCIFAR_TabFunctionAlphaBis_2000_01_03_06_5_09_02_02_N1000.npy',TotCoverage_AlphaBis)
#np.save('TotPowerCIFAR_TabFunctionAlphaBis_2000_01_03_06_5_09_02_02_N1000.npy',TotPower_AlphaBis)
#np.save('TotnPredictCIFAR_TabFunctionAlphaBis_2000_01_03_06_5_09_02_02_N1000.npy',TotnPredict_AlphaBis)

# $\lambda$ influence for $3$ different $\alpha$

In [ ]:
QuantileSimesTab=[QuantileSimesFunctionGenerator(AlphaViolin[0]),QuantileSimesFunctionGenerator(AlphaViolin[1]),QuantileSimesFunctionGenerator(AlphaViolin[2]),Quantile1_2,QuantileSimesFunctionGenerator(0.9)]

In [ ]:
StoreySimesTab=[AdaptiveSimesFunctionGenerator(AlphaViolin[0]),AdaptiveSimesFunctionGenerator(AlphaViolin[1]),AdaptiveSimesFunctionGenerator(AlphaViolin[2]),SimesStorey1_2,AdaptiveSimesFunctionGenerator(0.9)]

In [ ]:
TwoProcedure=np.append(QuantileSimesTab,StoreySimesTab)

In [ ]:
SimesBonfFisherTwoProcedure=np.append([SimesFunction,BonfFunction,FisherFunction],TwoProcedure)

In [ ]:
SimesBonfFisherTwoProcedure

In [ ]:
(TotCovCIFARD_2000_5_lambdaAlphaBIS,TotPowerCIFAR_2000_5_lambdaAlphaBIS,TotnPredictCIFAR_2000_5_lambdaAlphaBIS)=PowerAndCoverage3MethodsRandomized(AlphaBis,5,2000,XCIFARtestcal,yCIFARtestcal,ClassCalibratedpvalues,Classifieur_CIFAR,SimesBonfFisherTwoProcedure,AlphaLevel,3,N=500)


In [ ]:
#np.save('TotCovCIFAR_2000_5_lambdaAllProcess_AlphaBis_N500.npy',TotCovCIFARD_2000_5_lambdaAlphaBIS)
#np.save('TotPowerCIFAR_2000_5_lambdaAllProcess_AlphaBis_N500.npy',TotPowerCIFAR_2000_5_lambdaAlphaBIS)
#np.save('TotnPredictCIFAR_2000_5_lambdaAllProcess_AlphaBis_N500.npy',TotnPredictCIFAR_2000_5_lambdaAlphaBIS)

In [ ]:
(TotCovCIFARD_2000_5_lambdaTest,TotPowerCIFAR_2000_5_lambdaTest,TotnPredictCIFAR_2000_5_lambdaTest)=PowerAndCoverage3MethodsRandomized(AlphaViolin,5,2000,XCIFARtestcal,yCIFARtestcal,ClassCalibratedpvalues,Classifieur_CIFAR,SimesBonfFisherTwoProcedure,AlphaLevel,3,N=1000)


In [ ]:
#np.save('TotCovCIFAR_2000_5_lambdaAllProcess_N1000.npy',TotCovCIFARD_2000_5_lambdaTest)
#np.save('TotPowerCIFAR_2000_5_lambdaAllProcess_N1000.npy',TotPowerCIFAR_2000_5_lambdaTest)
#np.save('TotnPredictCIFAR_2000_5_lambdaAllProcess_N1000.npy',TotnPredictCIFAR_2000_5_lambdaTest)

In [ ]:
(TotCovCIFAR_2000_5_lambdaFull,TotPowerCIFAR_2000_5_lambdaFull,TotnPredictCIFAR_2000_5_lambdaFull)=PowerAndCoverage3MethodsRandomized(AlphaViolin,5,2000,XCIFARtestcal,yCIFARtestcal,FullCalibratedpvalues,Classifieur_CIFAR,TwoProcedure,AlphaLevel,3,N=500)
#np.save('TotCovCIFAR_2000_5_lambdaFull.npy',TotCovCIFAR_2000_5_lambdaFull)
#np.save('TotPowerCIFAR_2000_5_lambdaFull.npy',TotPowerCIFAR_2000_5_lambdaFull)
#np.save('TotnPredictCIFAR_2000_5_lambdaFull.npy',TotnPredictCIFAR_2000_5_lambdaFull)

In [ ]:
(TotCovUSPS_700_3_lambdaAllProcessClassAlphaBIS,TotPowerUSPS_700_3_lambdaAllProcessClassAlphaBIS,TotnPredictUSPS_700_3_lambdaAllProcessClassAlphaBIS)=PowerAndCoverage3MethodsRandomized(AlphaBis,3,700,XUSPSTestCal,yUSPSTestCal,ClassCalibratedpvalues,ClassifieurUSPS_SVC,SimesBonfFisherTwoProcedure,AlphaLevel,10,N=500)

In [ ]:
#np.save('TotCovUSPS_700_3_lambdaAllProcessClass_AlphaBis_N500.npy',TotCovUSPS_700_3_lambdaAllProcessClassAlphaBIS)
#np.save('TotPowerUSPS_700_3_lambdaAllProcessClass_AlphaBis_N500.npy',TotPowerUSPS_700_3_lambdaAllProcessClassAlphaBIS)
#np.save('TotnPredictUSPS_700_3_lambdaAllProcessClass_AlphaBis_N500.npy',TotnPredictUSPS_700_3_lambdaAllProcessClassAlphaBIS)

In [ ]:
(TotCovUSPS_700_3_lambdaFull,TotPowerUSPS_700_3_lambdaFull,TotnPredictUSPS_700_3_lambdaFull)=PowerAndCoverage3MethodsRandomized(AlphaViolin,3,700,XUSPSTestCal,yUSPSTestCal,FullCalibratedpvalues,ClassifieurUSPS_SVC,TwoProcedure,AlphaLevel,10,N=500)

In [ ]:
#np.save('TotCovUSPS_700_3_lambdaFull.npy',TotCovUSPS_700_3_lambdaFull)
#np.save('TotPowerUSPS_700_3_lambdaFull.npy',TotPowerUSPS_700_3_lambdaFull)
#np.save('TotnPredictUSPS_700_3_lambdaFull.npy',TotnPredictUSPS_700_3_lambdaFull)

# Batch Scores with the LRT

In [ ]:
Y=LRTBatchPrediction(0.1,1000,Classifieur_CIFAR,XCIFARcal,yCIFARcal.astype(int),XCIFARtest[:5],3)

In [ ]:
ScoreOpposite=Classifieur_CIFAR.predict_proba(XCIFARtestcal)


In [ ]:
(TotCoverage,TotPower,TotnPredict,Times)=PowerAndCoverage3MethodsRandomizedWithLRT(AlphaViolin,1000,5,2000,XCIFARtestcal,yCIFARtestcal,ClassCalibratedpvalues,Classifieur_CIFAR,TabFunction,AlphaLevel,3,N=1000)

In [ ]:

#np.save('TotCovCIFAR_2000_5_Proc_Bperm1000_N1000.npy',TotCoverage)
#np.save('TotPowerCIFAR_2000_5_Proc_Bperm1000_N1000.npy',TotPower)
#np.save('TotnPredictCIFAR_2000_5_Proc_Bperm1000_N1000.npy',TotnPredict)
#np.save('TotTimesCIFAR_2000_5_Proc_Bperm1000_N1000.npy',Times)

In [ ]:
(TotCoverageUSPS,TotPowerUSPS,TotnPredictUSPS,TimesUSPS)=PowerAndCoverage3MethodsRandomizedWithLRT(AlphaViolin,300,3,700,XUSPSTestCal,yUSPSTestCal,ClassCalibratedpvalues,ClassifieurUSPS_SVC,TabFunction,AlphaLevel,10,N=500)

In [ ]:
#np.save('TotCovUSPS_700_3_Proc_Bperm300_N500.npy',TotCoverageUSPS)
#np.save('TotPowerUSPS_700_3_Proc_Bperm300_N500.npy',TotPowerUSPS)
#np.save('TotnPredictUSPS_700_3_Proc_Bperm300_N500.npy',TotnPredictUSPS)
#np.save('TotTimesUSPS_700_3_Proc_Bperm300_N500.npy',TimesUSPS)

# Computing some bounds

In [ ]:
Index=[ 3316 , 8914 ,11507 , 5121 ,12781 ,10075  ,9866 , 2853 ,  493 , 3828]

In [ ]:
plt.figure(figsize=(9,3))
plt.gcf().subplots_adjust( wspace = 0, hspace = 0.5)
    
for i in range(10):
    plt.subplot(2,5,i+1)
    
    SeeCIFAR(XCIFARtestcal[Index[i]])
    plt.gca().set_title(LabelCIFAR[int(yCIFARtestcal[Index[i]])],fontsize=15)



plt.show()

In [ ]:
#####To Obtain Nices Pictures with LRT

#TabFunction Table of procedure function without LRT
#NameFunction Table with the name and final name is 'LRT'

def VizualisationPredictionSetwithLRT(alpha,pVal,ScoreOpposite,ycal,Bperm,I,Xtest,ytest,Ncal,TabFunction,NameFunction,GeneralQuantileFunction,nClass,Label,See,Compt='0',SaveFigs=False):
    nI=len(I)
    ntot=Ncal+nI
    nProc=len(TabFunction)
    
    
    YPredict=np.append(np.zeros(nI),NameFunction)
    for k in range(nI):
        YPredict[k]=Label[int(ytest[I[k]])]
    
    y=np.zeros(nI)
    NbCompt=0
    nBoucle=nClass**nI
    QuantileFunction=GeneralQuantileFunction(alpha,nClass,Ncal,nI,N=10**3)
    nPredict=0
    pValI=pVal[I]
    while NbCompt<nBoucle:
        SeeCurrenty=False
        
        
        # Do LRT
        (SeeyLRT,pValueLRT)=LRTIndicatorScores(alpha,Bperm,y,ycal,ScoreOpposite,nI,Ncal,ntot)
        if SeeyLRT:
            ToAdd=[]
            for k in range(nI):
                    
                ToAdd.append(Label[int(y[k])])
                
            for k in range(nProc):
                ToAdd.append("{:.3f}".format(min(TabFunction[k]( pValI[np.arange(nI),y.astype(int)]),1)))
            ToAdd.append("{:.3f}".format(min(pValueLRT,1)))
                
            YPredict=np.vstack([YPredict,ToAdd])
            nPredict+=1
        
        else:
            #Do Other Procedure
            for i in range(nProc):
                Function=TabFunction[i]

                if not SeeCurrenty and IndicatorPredictionSet(y, pValI,Function,QuantileFunction,nClass):
                    SeeCurrenty=True

                    ToAdd=[]
                    for k in range(nI):

                        ToAdd.append(Label[int(y[k])])
                    for k in range(nProc):
                        ToAdd.append("{:.3f}".format(min(TabFunction[k]( pValI[np.arange(nI),y.astype(int)]),1)))
                    ToAdd.append("{:.3f}".format(min(pValueLRT,1)))
                    
                    YPredict=np.vstack([YPredict,ToAdd])
                    nPredict+=1
                   


        y=CompteurYclass(y,nClass)

        NbCompt+=1
    
    
    
    
    
    plt.figure(figsize=(11,3))

    
    for i in range(nI):
        plt.subplot(1,nI,i+1)

        See(Xtest[I[i]])
        plt.gca().set_title(Label[int(ytest[I[i]])],fontsize=15)

    if SaveFigs:
        plt.savefig('Images'+Compt+'.pdf',format='pdf')

    plt.show()


    
    
    
    
    
    if nPredict>0:
        DataFramePval=pd.DataFrame(YPredict[1:],columns=YPredict[0])
        #DataFramePval.sort_values(by=['Bonf.','Simes'], inplace=True,ascending=[True,True])
        if SaveFigs:
            
            DataFramePval.to_latex(buf='pVal'+Compt+'.txt',index=False,formatters={"name": str.upper})


        return DataFramePval
    else:
        return  pd.DataFrame([],columns=YPredict)
    
    
    


In [ ]:
TabFunctionBis=[SimesFunction,BonfFunction,SimesStorey1_2,Quantile1_2,FisherFunction]
NameProcLRT=['Simes','Bonf.','Storey','Median','Fisher','LRT']


In [ ]:
nCalCIFAR=2000
J=np.random.choice(len(yCIFARtestcal),size=nCalCIFAR,replace=False)
XtestCurr=XCIFARtestcal[Index]
ytestCurr=yCIFARtestcal[Index]
XcalCurr=XCIFARtestcal[J].astype(int)
ycalCurr=yCIFARtestcal[J].astype(int)
    
(pValClassCIFAR,ncalClassCIFAR)=ClassCalibratedpvalues(Classifieur_CIFAR,nClassCIFAR,XcalCurr,ycalCurr,nCalCIFAR,XtestCurr,len(Index))
ScoreOppositeCIFAR=Classifieur_CIFAR.predict_proba(XCIFARtestcal[np.concatenate((J,Index))])

In [ ]:
ycalCurr

In [ ]:
Bperm=1000
UniI=np.arange(10).astype(int)

In [ ]:
pValCIFARonINDEX=VizualisationPredictionSetwithLRT(0.1,pValClassCIFAR,ScoreOppositeCIFAR,ycalCurr,Bperm,UniI,XtestCurr,ytestCurr,nCalCIFAR,TabFunctionBis,NameProcLRT,AlphaLevel,nClassCIFAR,LabelCIFAR,SeeCIFAR,Compt='3316_LRT_v10',SaveFigs=True)

In [ ]:
pValCIFARonINDEX

In [ ]:
VizualisationLowerBoundSet(0.1,pValCIFARonINDEX,10,NameProcLRT,LabelCIFAR,Compt='3316_v10',SaveFigs=True)